In [9]:
import pandas as pd
import numpy as np
import ast
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_data_cv = pd.read_csv('../../data/raw_data/datasets/cv_clean_V0')
df_data_job = pd.read_csv('../../data/raw_data/datasets/job_clean_V0')

In [3]:
# Convertir las cadenas de texto a lista de token real (pasamos de ["[...]"] a [[]])
df_data_cv['cv_tokens'] = df_data_cv['cv_tokens'].apply(lambda x: ast.literal_eval(x))
df_data_job['job_tokens'] = df_data_job['job_tokens'].apply(lambda x: ast.literal_eval(x))

**Carga de modelos**

In [5]:
model_cv=Word2Vec.load('../../data/raw_data/baseline_model/model_cv.bin')
model_job=Word2Vec.load('../../data/raw_data/baseline_model/model_job.bin')

In [8]:
#Imprimir longitud de palabras totales
print('Longitud de palabras totales modelo CV:',len(model_cv.wv))
print('Longitud de palabras totales modelo Jobs:',len(model_job.wv))

Longitud de palabras totales modelo CV: 345
Longitud de palabras totales modelo Jobs: 370


In [13]:
# Obtenemos las palabras coreespondientes a cada vector en ambos modelos
vocab_cv = model_cv.wv.key_to_index
vocab_job = model_job.wv.key_to_index

In [16]:
# Obtenemos los vectores de las palabras correspondientes
vector_cv_words = model_cv.wv[vocab_cv].reshape(1,-1)
vector_job_words= model_job.wv[vocab_job].reshape(1,-1)

In [23]:
# Obtenemos la longitud maximo de los vectores
max_len = max(len(vector_cv_words[0]), len(vector_job_words[0]))

In [32]:
# Rellena los vectores más cortos con ceros
def pad_vectors(vector_words, max_len):
    padded_vectors = []
    for vector in vector_words:
        if len(vector) < max_len:
            pad_width = max_len - len(vector)
            padded_vector = np.pad(vector, (0, pad_width), mode='constant', constant_values=0)
            padded_vectors.append(padded_vector)
        else:
            padded_vectors.append(vector)
    return np.array(padded_vectors)

In [33]:
vector_cv_words = pad_vectors(vector_cv_words, max_len)
vector_job_words = pad_vectors(vector_job_words, max_len)

In [38]:
# Calculo de similitud entre vectores
similarity_cosine = cosine_similarity(vector_cv_words, vector_job_words)

In [39]:
print(f"Similitud del coseno entre ambos modelos: {similarity_cosine[0][0]}")

Similitud del coseno entre ambos modelos: 0.8542696833610535


A nivel global, los textos encontrados en CV y Jobs son bastante parecidos.

Utilizaremos este indicador para futuros modelos mas avanzados, entrenados con deep learning.